In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
from IPython.display import display
from ipywidgets import FileUpload

Upload Gambar

In [ ]:
uploader = FileUpload(accept='image/*', multiple=False)
display(uploader)

Load Gambar dari Upload

In [ ]:
# Load image from uploader (untuk ipywidgets.FileUpload)
uploaded_files = uploader.value

if len(uploaded_files) == 0:
    print("Belum ada file yang di-upload!")
else:
    # Ambil file pertama
    file_info = uploaded_files[0]

    # Ambil konten biner
    content = file_info['content']

    # Konversi ke numpy array
    np_img = np.frombuffer(content, np.uint8)

    # Decode ke gambar OpenCV
    img = cv2.imdecode(np_img, cv2.IMREAD_COLOR)

    print("Gambar berhasil dimuat!")

    # Tampilkan
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Original Image")
    plt.axis("off")


## Preprocessing

In [ ]:
def preprocess_image(img):
    """Denoise dan konversi warna, tanpa resize (untuk pengukuran akurat)."""
    blur = cv2.GaussianBlur(img, (3, 3), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    return blur, hsv


OUTPUT PREPROCESSING

In [ ]:
blur, hsv = preprocess_image(img)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
axes[0].set_title("Original")
axes[0].axis("off")

axes[1].imshow(cv2.cvtColor(blur, cv2.COLOR_BGR2RGB))
axes[1].set_title("Gaussian Blur")
axes[1].axis("off")

axes[2].imshow(cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB))
axes[2].set_title("HSV")
axes[2].axis("off")

plt.show()

## Segmentasi

In [ ]:
def segment_image(hsv):

    lower_red1 = np.array([0, 40, 40])
    upper_red1 = np.array([15, 255, 255])
    lower_red2 = np.array([160, 40, 40])
    upper_red2 = np.array([180, 255, 255])

    lower_green = np.array([35, 40, 40])
    upper_green = np.array([90, 255, 255])

    lower_yellow = np.array([20, 40, 40])
    upper_yellow = np.array([45, 255, 255])

    mask_red = cv2.bitwise_or(cv2.inRange(hsv, lower_red1, upper_red1),
                              cv2.inRange(hsv, lower_red2, upper_red2))
    mask_green = cv2.inRange(hsv, lower_green, upper_green)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)

    mask = cv2.bitwise_or(mask_red, cv2.bitwise_or(mask_green, mask_yellow))

    bg_light = cv2.inRange(hsv, (0, 0, 160), (180, 60, 255))
    bg_dark = cv2.inRange(hsv, (0, 0, 0), (180, 100, 50))
    mask = cv2.bitwise_and(mask, cv2.bitwise_not(cv2.bitwise_or(bg_light, bg_dark)))

    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel, iterations=2)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel, iterations=1)

    edge_refine = cv2.inRange(hsv, (0, 0, 130), (180, 70, 255))
    edge_refine = cv2.GaussianBlur(edge_refine, (5, 5), 0)
    edge_refine = cv2.dilate(edge_refine, kernel, iterations=1)
    mask = cv2.bitwise_and(mask, cv2.bitwise_not(edge_refine))

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        filled_mask = np.zeros_like(mask)
        cv2.drawContours(filled_mask, [max(contours, key=cv2.contourArea)], -1, 255, -1)
        mask = filled_mask

    mask_blur = cv2.GaussianBlur(mask, (3, 3), 0)
    _, final_mask = cv2.threshold(mask_blur, 100, 255, cv2.THRESH_BINARY)

    segmented = cv2.bitwise_and(hsv, hsv, mask=final_mask)

    return segmented, final_mask

OUTPUT SEGMENTASI

In [ ]:
segmented, mask = segment_image(hsv)

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

axes[0].imshow(mask, cmap='gray')
axes[0].set_title("Segmentation Mask")
axes[0].axis("off")

axes[1].imshow(cv2.cvtColor(segmented, cv2.COLOR_HSV2RGB))
axes[1].set_title("Segmented Image")
axes[1].axis("off")

plt.show()

## Ekstraksi Fitur

In [ ]:
def extract_features(segmented_img, mask):

    if mask is None or np.count_nonzero(mask) == 0:
        return 0.0, 0.0, 0.0, 0.0

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return 0.0, 0.0, 0.0, 0.0

    c = max(contours, key=cv2.contourArea)
    x, y, w_box, h_box = cv2.boundingRect(c)

    pixel_per_cm = 102.0
    cm_per_pixel = 1.0 / pixel_per_cm

    length_cm = max(w_box, h_box) * cm_per_pixel * 0.9
    diameter_cm = min(w_box, h_box) * cm_per_pixel * 0.9

    ratio = length_cm / diameter_cm if diameter_cm > 0 else 0.0

    radius = diameter_cm / 2
    volume_cm3 = math.pi * (radius ** 2) * length_cm

    density = 0.22
    weight_est_g = density * volume_cm3
    weight_est_g *= 1.32

    return (
        round(length_cm, 3),
        round(diameter_cm, 3),
        round(weight_est_g, 3),
        round(ratio, 4)
    )


OUTPUT EKSTRAKSI FITUR

In [ ]:
length, diameter, weight, ratio = extract_features(segmented, mask)

print("=== HASIL EKSTRAKSI FITUR ===")
print(f"Panjang (cm)   : {length}")
print(f"Diameter (cm)  : {diameter}")
print(f"Berat (g)      : {weight}")
print(f"Rasio L/D      : {ratio}")
